In [421]:
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

# Data Preparation

## Dataset Aggregation

In [422]:
trips = pd.read_csv('trips_raw_data.csv')

In [423]:
trips.head()

,Unnamed: 0,Trip Id,Subscription Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,...,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Hmdx,Wind Chill,Weather
0,58,712441,NaN,274,7006.0,2017-01-01 00:03:00-05:00,Bay St / College St (East Side),7021.0,2017-01-01 00:08:00-05:00,Bay St / Albert St,...,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
1,59,712442,NaN,538,7046.0,2017-01-01 00:03:00-05:00,Niagara St / Richmond St W,7147.0,2017-01-01 00:12:00-05:00,King St W / Fraser Ave,...,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
2,60,712443,NaN,992,7048.0,2017-01-01 00:05:00-05:00,Front St W / Yonge St (Hockey Hall of Fame),7089.0,2017-01-01 00:22:00-05:00,Church St / Wood St,...,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
3,61,712444,NaN,1005,7177.0,2017-01-01 00:09:00-05:00,East Liberty St / Pirandello St,7202.0,2017-01-01 00:26:00-05:00,Queen St W / York St (City Hall),...,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
4,62,712445,NaN,645,7203.0,2017-01-01 00:14:00-05:00,Bathurst St/Queens Quay(Billy Bishop Airport),7010.0,2017-01-01 00:25:00-05:00,King St W / Spadina Ave,...,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN


In [424]:
trips['Start Time'] = pd.DatetimeIndex(trips['Start Time'])

Our model will be focused on predicting BikeShare trips for a "normal" year. Whether that future years represent a normal year is unknown, but we also can't assume any of the trends in 2020 will continue.

We will limit the model to look only at 2019 data, from January 1st to December 31st.

In [425]:
trips = trips[trips['Start Time'].dt.year == 2019]

# Feature Engineering

In [505]:
from sklearn.model_selection import train_test_split

In [506]:
train, test = train_test_split(trips, train_size=0.7, 
                               test_size=0.3, random_state=0)
val, test = train_test_split(test, train_size=0.5,
                             test_size=0.5, random_state=0)

## Chosen Parameters

* Wind Speed
* Hour of Day
* Month
* Weekend vs Weekday - if weekend or statutory holiday, then 1, otherwise 0
* Temperature - binned with this distribution [-30,0,10,20,30,40]
* Precipitation - if precipitation present, then 1, otherwise 0

In [507]:
def aggregate_trips(df):
    df['Start Time'] = df['Start Time'].dt.floor('H')
    
    df_hourly = df.groupby('Start Time').agg({'Trip Id':'count', 'Weather':'first',
                                              'Wind Spd (km/h)': 'first', 'Temp (°C)': 'first'})
    df_hourly = df_hourly.reset_index()
    df_hourly = df_hourly.rename(columns = {'Trip Id': 'Trips', 'Wind Spd (km/h)':'Wind', 'Temp (°C)': 'Temp'})
    
    return df_hourly

In [508]:
def weather_features(df):
    
    df['Temp'] = np.digitize(df.loc[:, 'Temp'], [-30,0,10,20,30,40])
    df['Weather'] = np.where(df['Weather'].isna(), 0, 1)
    
    return df

In [509]:
!pip install holidays

In [510]:
from datetime import date
import holidays

In [511]:
on_hol = holidays.CA(prov = 'ON', years = 2019)
for i in on_hol:
    print(i)

2019-01-01
2019-02-18
2019-04-19
2019-05-20
2019-07-01
2019-08-05
2019-09-02
2019-10-14
2019-12-25
2019-12-26


In [512]:
def temporal_features(df):
    
    df['Month'] = df['Start Time'].dt.month
    df['Hour'] = df['Start Time'].dt.hour
    df['dow'] = np.where(df['Start Time'].dt.dayofweek > 4, 1, 0)
    df['dow'] = np.where(df['Start Time'].dt.date.isin(on_hol), 1, 0)
    
    return df

In [513]:
train_hourly = aggregate_trips(train)

In [514]:
train_hourly.head()

,Start Time,Trips,Weather,Wind,Temp
0,2019-01-01 00:00:00-05:00,15,"Rain,Fog",4.0,5.1
1,2019-01-01 01:00:00-05:00,21,"Rain,Fog",28.0,5.9
2,2019-01-01 02:00:00-05:00,16,"Rain,Fog",34.0,3.3
3,2019-01-01 03:00:00-05:00,7,NaN,28.0,2.8
4,2019-01-01 04:00:00-05:00,6,NaN,28.0,2.8


In [515]:
train_weather = weather_features(train_hourly)

In [516]:
train_weather.head()

,Start Time,Trips,Weather,Wind,Temp
0,2019-01-01 00:00:00-05:00,15,1,4.0,2
1,2019-01-01 01:00:00-05:00,21,1,28.0,2
2,2019-01-01 02:00:00-05:00,16,1,34.0,2
3,2019-01-01 03:00:00-05:00,7,0,28.0,2
4,2019-01-01 04:00:00-05:00,6,0,28.0,2


In [517]:
train_dt = temporal_features(train_weather)
train_dt.head()

,Start Time,Trips,Weather,Wind,Temp,Month,Hour,dow
0,2019-01-01 00:00:00-05:00,15,1,4.0,2,1,0,1
1,2019-01-01 01:00:00-05:00,21,1,28.0,2,1,1,1
2,2019-01-01 02:00:00-05:00,16,1,34.0,2,1,2,1
3,2019-01-01 03:00:00-05:00,7,0,28.0,2,1,3,1
4,2019-01-01 04:00:00-05:00,6,0,28.0,2,1,4,1


In [684]:
from sklearn.preprocessing import MinMaxScaler 

def process_data(df):
    
    df = aggregate_trips(df)
    df = weather_features(df)
    df = temporal_features(df)
    
    df = df.drop(columns = ['Start Time'])
    y = df['Trips']
    
    cat_features = ['Temp', 'Month', 'Hour']
    
    categoricals = [pd.get_dummies(df[s], prefix=s, drop_first=True) for s in cat_features]
    
    scaler = MinMaxScaler()
    scaler.fit(df[['Wind']])
    
    scaled = df[['Wind']]
    
    scaled.iloc[:, :] = scaler.transform(scaled) 
    
    x = pd.concat([scaled] + categoricals, axis=1)
    
    return x, y

In [685]:
x_train, y_train = process_data(train)

In [679]:
x_train.head()

,Wind,Temp_2,Temp_3,Temp_4,Temp_5,Temp_6,Month_2,Month_3,Month_4,Month_5,...,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23
0,0.058824,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.411765,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.500000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.411765,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.411765,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [665]:
train.isnull().sum()

Unnamed: 0                   0
Trip Id                      0
Subscription Id              0
Trip Duration                0
Start Station Id             0
Start Time                   0
Start Station Name           0
End Station Id               0
End Time                     0
End Station Name             0
Bike Id                      0
User Type                    0
merge_time                   0
Date/Time                    0
Temp (°C)                18423
Dew Point Temp (°C)      23255
Rel Hum (%)              23255
Wind Dir (10s deg)      108599
Wind Spd (km/h)              0
Visibility (km)           4699
Stn Press (kPa)          18579
Hmdx                   1223570
Wind Chill             1452203
Weather                1472929
dtype: int64

Filling Wind Speed NA with average speed in both Train and Validation

In [666]:
train['Wind Spd (km/h)'].fillna(train["Wind Spd (km/h)"].median(skipna=True), inplace=True)
val['Wind Spd (km/h)'].fillna(val["Wind Spd (km/h)"].median(skipna=True), inplace=True)

# Model Fitting
In this section, we will be using linear regression model to predict ridership 

First defining the RMSE calculation

In [668]:
def rmse(errors):
    return np.sqrt(np.sum(errors ** 2)/len(errors))

## Constant RMSE
Next, we will establish a baseline model to compare, which will be our constant RMSE 

In [669]:
train_constant = aggregate_trips(train)
val_constant = aggregate_trips(val)

trip_mean_train = train_constant['Trips'].mean()
constant_rmse = rmse(val_constant['Trips']- trip_mean_train)
print(constant_rmse)

153.84957716049942


## Simple RMSE
Once a baseline is established, we will now create a simple model that predicts trips with only one variable, in this case wind speed without any scaling, we will also fill NA with average in wind speed 

In [670]:
train_hourly = aggregate_trips(train)
train_weather = weather_features(train_hourly)
val_hourly = aggregate_trips(val)
val_weather = weather_features(val_hourly)
val_weather.isnull().sum()

Start Time    0
Trips         0
Weather       0
Wind          0
Temp          0
dtype: int64

In [671]:
def select_columns(data, *columns):
    return data.loc[:, columns]

def process_data_w(data):
    data = select_columns(data, 
                          'Trips', 
                          'Wind')
    
    # Return predictors and response variables separately
    X = data.drop(['Trips'], axis = 1)
    y = data.loc[:, 'Trips']
    
    return X, y

X_train, y_train = process_data_w(train_weather)

X_val, y_val = process_data_w(val_weather)

In [672]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression(fit_intercept=True)

linear_model.fit(X_train, y_train)
y_predicted = linear_model.predict(X_val)

simple_rmse = rmse(y_val-y_predicted)
print(simple_rmse)

154.1379582013924


## Linear RMSE
Now we will derive a linear regression model using all the features provided above 

In [673]:
X_train, y_train = process_data(train)
X_val, y_val = process_data(val)

In [674]:
linear_model.fit(X_train, y_train)
y_predicted = linear_model.predict(X_val)

linear_rmse = rmse(y_val-y_predicted)
print(linear_rmse)

201.6345218029761


Removing temperature from analysis 

In [675]:
def temp_remove (df):
    df = df.drop(['Temp_2','Temp_3','Temp_4','Temp_5','Temp_6'], axis=1)
    
    return df  

In [676]:
X_train, y_train = process_data(train)
X_train = temp_remove(X_train)
X_val, y_val = process_data(val)
X_val = temp_remove(X_val)

In [677]:
linear_model.fit(X_train, y_train)
y_predicted = linear_model.predict(X_val)

linear_notemp_rmse = rmse(y_val-y_predicted)
print(linear_notemp_rmse)

200.51437310687422


Remove month only 

In [592]:
def mon_remove (df):
    df = df.drop(['Month_2','Month_3','Month_4','Month_5','Month_6','Month_7','Month_8','Month_9','Month_10','Month_11','Month_12'], axis=1)
    
    return df  

In [593]:
X_train, y_train = process_data(train)
X_train = mon_remove(X_train)
X_val, y_val = process_data(val)
X_val = mon_remove(X_val)

In [594]:
linear_model.fit(X_train, y_train)
y_predicted = linear_model.predict(X_val)

linear_nomon_rms = rmse(y_val-y_predicted)
print(linear_nomon_rms)

201.0107140307012


Remove month & temperature 

In [595]:
X_train, y_train = process_data(train)
X_train = temp_remove(X_train)
X_train = mon_remove(X_train)
X_val, y_val = process_data(val)
X_val = temp_remove(X_val)
X_val = mon_remove(X_val)

In [596]:
linear_model.fit(X_train, y_train)
y_predicted = linear_model.predict(X_val)

linear_nomontemp_rmse = rmse(y_val-y_predicted)
print(linear_nomontemp_rmse)

193.48711941609852


Remove Hour 

In [597]:
hour = X_train.columns.tolist()
hour = hour[-23:]
print(hour)

['Hour_1', 'Hour_2', 'Hour_3', 'Hour_4', 'Hour_5', 'Hour_6', 'Hour_7', 'Hour_8', 'Hour_9', 'Hour_10', 'Hour_11', 'Hour_12', 'Hour_13', 'Hour_14', 'Hour_15', 'Hour_16', 'Hour_17', 'Hour_18', 'Hour_19', 'Hour_20', 'Hour_21', 'Hour_22', 'Hour_23']


In [598]:
def hour_remove (df):
    df = df.drop(labels=hour, axis=1)
    
    return df  

In [599]:
X_train, y_train = process_data(train)
X_train = hour_remove(X_train)
X_val, y_val = process_data(val)
X_val = hour_remove(X_val)

In [600]:
linear_model.fit(X_train, y_train)
y_predicted = linear_model.predict(X_val)

linear_nohour_rmse= rmse(y_val-y_predicted)
print(linear_nohour_rmse)

171.1057239629335


Remove month and hour 

In [601]:
X_train, y_train = process_data(train)
X_train = hour_remove(X_train)
X_train = mon_remove(X_train)
X_val, y_val = process_data(val)
X_val = hour_remove(X_val)
X_val = mon_remove(X_val)

In [602]:
linear_model.fit(X_train, y_train)
y_predicted = linear_model.predict(X_val)

linear_nohourmon_rmse = rmse(y_val-y_predicted)
print(linear_nohourmon_rmse)

170.1381152929613


Remove hour and temperature 

In [603]:
X_train, y_train = process_data(train)
X_train = hour_remove(X_train)
X_train = temp_remove(X_train)
X_val, y_val = process_data(val)
X_val = hour_remove(X_val)
X_val = temp_remove(X_val)

In [604]:
linear_model.fit(X_train, y_train)
y_predicted = linear_model.predict(X_val)

linear_nohourtemp_rmse= rmse(y_val-y_predicted)
print(linear_nohourtemp_rmse)

165.3507756314079


## Time Series Cross Validation

In [605]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.base import clone

In [686]:
# this code is from assignment 6, except for TimeSeriesSplit

def cross_validate_rmse(model, X, y):
    
    # Setup
    model = clone(model)
    time_split = TimeSeriesSplit(n_splits=5) 
    rmse_values = []
    
    for train_index, val_index in time_split.split(X):
        
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_val)   
        
        
        rmse_values.append(rmse(y_val - y_predicted))
        
    return rmse_values

First applying on simple model 

In [687]:
train_hourly = aggregate_trips(train)
train_weather = weather_features(train_hourly)
val_hourly = aggregate_trips(val)
val_weather = weather_features(val_hourly)

X_train, y_train = process_data_w(train_weather)

X_val, y_val = process_data_w(val_weather)


In [688]:
simple_rmse_cv = cross_validate_rmse(model=LinearRegression(fit_intercept=True), X=X_train, y=y_train)
print('Cross-validation Simple RMSE scores: {}'.format(simple_rmse_cv))
print('Cross-validation Simple RMSE scores mean: {}'.format(np.mean(simple_rmse_cv)))
print('Cross-validation Simple RMSE scores std: {}'.format(np.std(simple_rmse_cv)))

Cross-validation Simple RMSE scores: [177.40845395827333, 239.72550982521943, 329.93524556529275, 246.62161541137726, 179.92262053532767]
Cross-validation Simple RMSE scores mean: 234.7226890590981
Cross-validation Simple RMSE scores std: 55.71357578867606


Next linear model is used 

In [689]:
X_train, y_train = process_data(train)
X_val, y_val = process_data(val)

In [690]:
linear_rmse_cv = cross_validate_rmse(model=LinearRegression(fit_intercept=True), X=X_train, y=y_train)
print('Cross-validation Linear RMSE scores: {}'.format(linear_rmse_cv))
print('Cross-validation Linear RMSE scores mean: {}'.format(np.mean(linear_rmse_cv)))
print('Cross-validation Linear RMSE scores std: {}'.format(np.std(linear_rmse_cv)))

Cross-validation Linear RMSE scores: [1808146244370513.0, 1093056485477305.6, 14338347399331.71, 200.15109765920283, 144.3271232494607]
Cross-validation Linear RMSE scores mean: 583108215449498.9
Cross-validation Linear RMSE scores std: 743544881639008.6


Remove month only 

In [691]:
X_train, y_train = process_data(train)
X_train = temp_remove(X_train)
X_val, y_val = process_data(val)
X_val = temp_remove(X_val)

In [692]:
notemp_rmse_cv = cross_validate_rmse(model=LinearRegression(fit_intercept=True), X=X_train, y=y_train)
print('Cross-validation Linear RMSE scores: {}'.format(notemp_rmse_cv))
print('Cross-validation Linear RMSE scores mean: {}'.format(np.mean(notemp_rmse_cv)))
print('Cross-validation Linear RMSE scores std: {}'.format(np.std(notemp_rmse_cv)))

Cross-validation Linear RMSE scores: [4908140802598774.0, 192143639304351.28, 922196253187145.5, 247389762824762.2, 334980127860576.3]
Cross-validation Linear RMSE scores mean: 1320970117155121.5
Cross-validation Linear RMSE scores std: 1812501827551098.8


# KFOLD Validation

In [693]:
from sklearn.model_selection import KFold
from sklearn.base import clone

def cross_validate_rmse_k(model, X, y):
    
    # Setup
    model = clone(model)
    five_fold = KFold(n_splits=5)
    rmse_values = []
    
    # Iterature thought cv-folds
    for train_index, val_index in five_fold.split(X):
        
        # Write your code here.
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]        
        
        # Fit model
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_val)     
        
        # Append RMSE scores
        rmse_values.append(rmse(y_val-y_predicted))
    
    return rmse_values

In [694]:
train_hourly = aggregate_trips(train)
train_weather = weather_features(train_hourly)
val_hourly = aggregate_trips(val)
val_weather = weather_features(val_hourly)

X_train, y_train = process_data_w(train_weather)

X_val, y_val = process_data_w(val_weather)

In [695]:
simple_rmse_cvk = cross_validate_rmse_k(model=LinearRegression(fit_intercept=True), X=X_train, y=y_train)
print('Cross-validation Simple RMSE scores: {}'.format(simple_rmse_cvk))
print('Cross-validation Simple RMSE scores mean: {}'.format(np.mean(simple_rmse_cvk)))
print('Cross-validation Simple RMSE scores std: {}'.format(np.std(simple_rmse_cvk)))

Cross-validation Simple RMSE scores: [189.1512524810011, 184.11516288454936, 263.78315362985734, 295.9922328804614, 185.6288454184065]
Cross-validation Simple RMSE scores mean: 223.7341294588551
Cross-validation Simple RMSE scores std: 46.99533681424505


In [696]:
X_train, y_train = process_data(train)
X_val, y_val = process_data(val)

In [697]:
linear_rmse_cv = cross_validate_rmse_k(model=LinearRegression(fit_intercept=True), X=X_train, y=y_train)
print('Cross-validation Linear RMSE scores: {}'.format(linear_rmse_cv))
print('Cross-validation Linear RMSE scores mean: {}'.format(np.mean(linear_rmse_cv)))
print('Cross-validation Linear RMSE scores std: {}'.format(np.std(linear_rmse_cv)))

Cross-validation Linear RMSE scores: [248816604697114.56, 134.7998276737242, 30350274203914.41, 221.043772589008, 149.0571788205295]
Cross-validation Linear RMSE scores mean: 55833375780306.77
Cross-validation Linear RMSE scores std: 97204951172384.44
